In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

In [0]:
#mount and get data
storageAccount = "gen10datafund2205"
storageContainer = "school-finances-container"
clientSecret = 
clientid = 
mount_point = "/mnt/julianngroglio/school-finance"
    
    
configs = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs)

/mnt/julianngroglio/school-finance has been unmounted.
Out[2]: True

In [0]:
%fs

ls /mnt/julianngroglio/school-finance

path,name,size,modificationTime
dbfs:/mnt/julianngroglio/school-finance/census-clean/,census-clean/,0,1660059215000
dbfs:/mnt/julianngroglio/school-finance/census-data,census-data,7255007,1659469854000
dbfs:/mnt/julianngroglio/school-finance/demo-clean/,demo-clean/,0,1660059217000
dbfs:/mnt/julianngroglio/school-finance/demographic-factors,demographic-factors,93703,1660055843000
dbfs:/mnt/julianngroglio/school-finance/grad-clean/,grad-clean/,0,1660059216000
dbfs:/mnt/julianngroglio/school-finance/grad-rate,grad-rate,166631,1659993188000
dbfs:/mnt/julianngroglio/school-finance/name-changes,name-changes,20054,1660155004000


In [0]:
censusdf = spark.read.format('csv').options(header='true', inferschema='true').load("/mnt/julianngroglio/school-finance/census-data")
#display(censusdf)

In [0]:
census_cleaned = censusdf[['STATE','NAME', 'TSTREV', 'TLOCREV', 'C05', 'E17', 'T06', 'Z32', 'TOTALREV', 'TFEDREV', 'C16', 'C04', 'C08', 'TOTALEXP', 'TCURELSC', 'TCURINST', 'TCURSSVC', 'E07', 'E08', 'E09']]

In [0]:
census_cleaned = census_cleaned.withColumnRenamed("TSTREV","Total_revenue_state")
census_cleaned = census_cleaned.withColumnRenamed("TLOCREV","Total_revenue_from_local_sources")
census_cleaned = census_cleaned.withColumnRenamed("C05","SPED_rev_state")
census_cleaned = census_cleaned.withColumnRenamed("E17","COE_pupil_support")
census_cleaned = census_cleaned.withColumnRenamed("T06","Property_tax_revenue")
census_cleaned = census_cleaned.withColumnRenamed("Z32","Total_salaries_and_wages")
census_cleaned = census_cleaned.withColumnRenamed("TOTALREV","Total_revenue")
census_cleaned = census_cleaned.withColumnRenamed("TFEDREV","Total_federal_revenue")
census_cleaned = census_cleaned.withColumnRenamed("C16","FEDREV_math_science_teach_quality")
census_cleaned = census_cleaned.withColumnRenamed("C04","STATEREV_staff_improvement_programs")
census_cleaned = census_cleaned.withColumnRenamed("C08","STATEREV_gifted_and_talented_programs")
census_cleaned = census_cleaned.withColumnRenamed("TCURELSC","Total_current_spending_on_programs")
census_cleaned = census_cleaned.withColumnRenamed("TOTALEXP","Total_expenditure")
census_cleaned = census_cleaned.withColumnRenamed("TCURINST","Current_spending_on_instruction")
census_cleaned = census_cleaned.withColumnRenamed("E17","Expenditure_pupil_support")
census_cleaned = census_cleaned.withColumnRenamed("E07","Expenditure_instructional_staff_support")
census_cleaned = census_cleaned.withColumnRenamed("TCURSSVC","Total_expenditure_support_services")
census_cleaned = census_cleaned.withColumnRenamed("E08","Expenditure_gen_administration")
census_cleaned = census_cleaned.withColumnRenamed("E09","Expenditure_school_administration")
census_cleaned = census_cleaned.withColumnRenamed("STATE","StateID")
census_cleaned = census_cleaned.withColumnRenamed("NAME","School_district")

In [0]:
census_cleaned = census_cleaned.where(census_cleaned.StateID == '33')
display(census_cleaned)

StateID,School_district,Total_revenue_state,Total_revenue_from_local_sources,SPED_rev_state,COE_pupil_support,Property_tax_revenue,Total_salaries_and_wages,Total_revenue,Total_federal_revenue,FEDREV_math_science_teach_quality,STATEREV_staff_improvement_programs,STATEREV_gifted_and_talented_programs,Total_expenditure,Total_current_spending_on_programs,Current_spending_on_instruction,Total_expenditure_support_services,Expenditure_instructional_staff_support,Expenditure_gen_administration,Expenditure_school_administration
33,BUFFALO CITY SCH DIST,830603,145801,108671,36598,0,361669,1063839,87435,0,0,0,1002588,894051,596399,255087,33541,4528,42617
33,ROCHESTER CITY SCH DIST,698695,136369,96350,40601,0,398096,908105,73041,0,0,0,1084531,849263,548635,273442,29439,5076,48430
33,NEW YORK CITY SCHOOL DISTRICT,11852988,19976042,1571872,233803,0,12535679,33788513,1959483,0,0,0,33799629,30024166,23100440,6346293,266295,87476,1213363
33,SYRACUSE CITY SCH DIST,384640,76934,52520,20380,0,239781,515531,53957,0,0,0,561015,463993,298635,146957,19320,3229,21553
33,YONKERS CITY SCH DIST,392963,234118,47788,25555,0,337273,670050,42969,0,0,0,673848,621402,440768,167238,8600,2717,40162
33,BERNE KNOX-WESTERLO CENTRAL SCH DIST,11602,10098,1756,697,9764,10605,22329,629,0,0,0,22825,19785,12277,7142,744,649,895
33,BETHLEHEM CENTRAL SCH,34069,67455,5320,5234,60826,52610,103128,1604,0,0,0,97197,86587,53141,32075,3772,1667,4963
33,RAVENA COEYMANS SELKIRK CENTRAL SCH DIST,21653,26748,4793,1960,22973,24785,49793,1392,0,0,0,51022,43408,28292,14296,1698,920,1363
33,COHOES CITY SCH DIST,26322,15794,3866,2170,13614,22796,45204,3088,0,0,0,44261,42528,28383,12718,1719,869,2074
33,SOUTH COLONIE CENTRAL SCH DIST,32321,69185,4717,3507,64173,62033,104825,3319,0,0,0,103093,97743,69559,26213,3359,1565,3866


In [0]:
from pyspark.sql import functions as sf
census_cleaned_sp =census_cleaned.withColumn('School_district',sf.concat(sf.col('School_district'),sf.lit(' ')))


In [0]:
census_cleaned1 = census_cleaned_sp.withColumn('School_district', regexp_replace('School_district', ' SCH ', ' SCHOOL '))

In [0]:
#display(census_cleaned1)

In [0]:
census_cleaned2 = census_cleaned1.withColumn('School_district', regexp_replace('School_district', ' DIST ', ' DISTRICT '))

In [0]:
#TAKE OUT SPACES
census_cleaned2 = census_cleaned2.withColumn('School_district', rtrim(census_cleaned2.School_district))

In [0]:
schools_we_have = ["ADDISON CENTRAL SCHOOL DISTRICT",
"ADIRONDACK CENTRAL SCHOOL DISTRICT",
"AFTON CENTRAL SCHOOL DISTRICT",
"AKRON CENTRAL SCHOOL DISTRICT",
"ALBANY CITY SCHOOL DISTRICT",
"ALBION CENTRAL SCHOOL DISTRICT"
"ALDEN CENTRAL SCHOOL DISTRICT",
"ALEXANDRIA CENTRAL SCHOOL DISTRICT",
"AMHERST CENTRAL SCHOOL DISTRICT",
"ANDES CENTRAL SCHOOL DISTRICT",
"ANDOVER CENTRAL SCHOOL DISTRICT",
"ARGYLE CENTRAL SCHOOL DISTRICT",
"ARKPORT CENTRAL SCHOOL DISTRICT",
"ARLINGTON CENTRAL SCHOOL DISTRICT",
"ATTICA CENTRAL SCHOOL DISTRICT",
"AUBURN CITY SCHOOL DISTRICT",
"AUSABLE VALLEY CENTRAL SCHOOL DISTRICT",
"AVOCA CENTRAL SCHOOL DISTRICT",
"AVON CENTRAL SCHOOL DISTRICT",
"BABYLON UNION FREE SCHOOL DISTRICT",
"BALDWIN UNION FREE SCHOOL DISTRICT",
"BARKER CENTRAL SCHOOL DISTRICT",
"BATAVIA CITY SCHOOL DISTRICT",
"BATH CENTRAL SCHOOL DISTRICT",
"BAY SHORE UNION FREE SCHOOL DISTRICT",
"BEACON CITY SCHOOL DISTRICT",
"BEAVER RIVER CENTRAL SCHOOL DISTRICT",
"BEDFORD CENTRAL SCHOOL DISTRICT",
"BELFAST CENTRAL SCHOOL DISTRICT",
"BERLIN CENTRAL SCHOOL DISTRICT",
"BINGHAMTON CITY SCHOOL DISTRICT",
"BOLIVAR-RICHBURG CENTRAL SCHOOL DISTRICT",
"BOLTON CENTRAL SCHOOL DISTRICT",
"BRADFORD CENTRAL SCHOOL DISTRICT",
"BRASHER FALLS CENTRAL SCHOOL DISTRICT",
"BREWSTER CENTRAL SCHOOL DISTRICT",
"BRIGHTON CENTRAL SCHOOL DISTRICT",
"BROCKPORT CENTRAL SCHOOL DISTRICT",
"BROCTON CENTRAL SCHOOL DISTRICT",
"BRONXVILLE UNION FREE SCHOOL DISTRICT",
"BROOKFIELD CENTRAL SCHOOL DISTRICT",
"BROOKHAVEN-COMSEWOGUE UNION FREE SCHOOL DISTRICT",
"BUFFALO CITY SCHOOL DISTRICT",
"CAIRO-DURHAM CENTRAL SCHOOL DISTRICT",
"CAMBRIDGE CENTRAL SCHOOL DISTRICT",
"CAMDEN CENTRAL SCHOOL DISTRICT",
"CANASERAGA CENTRAL SCHOOL DISTRICT",
"CANASTOTA CENTRAL SCHOOL DISTRICT",
"CANDOR CENTRAL SCHOOL DISTRICT",
"CANTON CENTRAL SCHOOL DISTRICT",
"CARMEL CENTRAL SCHOOL DISTRICT",
"CARTHAGE CENTRAL SCHOOL DISTRICT",
"CATO-MERIDIAN CENTRAL SCHOOL DISTRICT",
"CENTRAL SQUARE CENTRAL SCHOOL DISTRICT",
"CHAPPAQUA CENTRAL SCHOOL DISTRICT",
"CHATHAM CENTRAL SCHOOL DISTRICT",
"CHAUTAUQUA LAKE CENTRAL SCHOOL DISTRICT",
"HAZY UNION FREE SCHOOL DISTRICT",
"CHEEKTOWAGA CENTRAL SCHOOL DISTRICT",
"CHEEKTOWAGA-MARYVALE UNION FREE SCHOOL DISTRICT",
"CHEEKTOWAGA-SLOAN UNION FREE SCHOOL DISTRICT",
"CHENANGO FORKS CENTRAL SCHOOL DISTRICT",
"CHENANGO VALLEY CENTRAL SCHOOL DISTRICT",
"CLARENCE CENTRAL SCHOOL DISTRICT",
"CLARKSTOWN CENTRAL SCHOOL DISTRICT",
"CLIFTON-FINE CENTRAL SCHOOL DISTRICT",
"CLINTON CENTRAL SCHOOL DISTRICT",
"CLYDE-SAVANNAH CENTRAL SCHOOL DISTRICT",
"CLYMER CENTRAL SCHOOL DISTRICT",
"COHOES CITY SCHOOL DISTRICT",
"COLTON-PIERREPONT CENTRAL SCHOOL DISTRICT",
"COOPERSTOWN CENTRAL SCHOOL DISTRICT",
"COPENHAGEN CENTRAL SCHOOL DISTRICT",
"CORINTH CENTRAL SCHOOL DISTRICT",
"CORNING CITY SCHOOL DISTRICT",
"CORNWALL CENTRAL SCHOOL DISTRICT",
"CORTLAND CITY SCHOOL DISTRICT",
"DANSVILLE CENTRAL SCHOOL DISTRICT",
"DEPEW UNION FREE SCHOOL DISTRICT",
"DEPOSIT CENTRAL SCHOOL DISTRICT",
"DERUYTER CENTRAL SCHOOL DISTRICT",
"DOBBS FERRY UNION FREE SCHOOL DISTRICT",
"DOWNSVILLE CENTRAL SCHOOL DISTRICT",
"DRYDEN CENTRAL SCHOOL DISTRICT",
"DUANESBURG CENTRAL SCHOOL DISTRICT",
"DUNDEE CENTRAL SCHOOL DISTRICT",
"DUNKIRK CITY SCHOOL DISTRICT",
"EAST AURORA UNION FREE SCHOOL DISTRICT",
"EAST BLOOMFIELD CENTRAL SCHOOL DISTRICT",
"EAST GREENBUSH CENTRAL SCHOOL DISTRICT",
"EAST IRONDEQUOIT CENTRAL SCHOOL DISTRICT",
"EAST MEADOW UNION FREE SCHOOL DISTRICT",
"EAST ROCKAWAY UNION FREE SCHOOL DISTRICT",
"EASTCHESTER UNION FREE SCHOOL DISTRICT",
"EDEN CENTRAL SCHOOL DISTRICT",
"EDWARDS-KNOX CENTRAL SCHOOL DISTRICT",
"ELBA CENTRAL SCHOOL DISTRICT",
"ELDRED CENTRAL SCHOOL DISTRICT",
"ELLENVILLE CENTRAL SCHOOL DISTRICT",
"ELMIRA CITY SCHOOL DISTRICT",
"FABIUS-POMPEY CENTRAL SCHOOL DISTRICT",
"FAIRPORT CENTRAL SCHOOL DISTRICT",
"FALCONER CENTRAL SCHOOL DISTRICT",
"FALLSBURG CENTRAL SCHOOL DISTRICT",
"FARMINGDALE UNION FREE SCHOOL DISTRICT",
"FAYETTEVILLE-MANLIUS CENTRAL SCHOOL DISTRICT",
"FILLMORE CENTRAL SCHOOL DISTRICT",
"FISHERS ISLAND UNION FREE SCHOOL DISTRICT",
"FLORIDA UNION FREE SCHOOL DISTRICT",
"FORT ANN CENTRAL SCHOOL DISTRICT",
"FRANKLIN CENTRAL SCHOOL DISTRICT",
"FREEPORT UNION FREE SCHOOL DISTRICT",
"FREWSBURG CENTRAL SCHOOL DISTRICT",
"FRIENDSHIP CENTRAL SCHOOL DISTRICT",
"FRONTIER CENTRAL SCHOOL DISTRICT",
"FULTON CITY SCHOOL DISTRICT",
"GALWAY CENTRAL SCHOOL DISTRICT",
"GANANDA CENTRAL SCHOOL DISTRICT",
"GENESEO CENTRAL SCHOOL DISTRICT",
"GENEVA CITY SCHOOL DISTRICT",
"GLEN COVE CITY SCHOOL DISTRICT",
"GLENS FALLS CITY SCHOOL DISTRICT",
"GOSHEN CENTRAL SCHOOL DISTRICT",
"GOWANDA CENTRAL SCHOOL DISTRICT",
"GRANVILLE CENTRAL SCHOOL DISTRICT",
"GREECE CENTRAL SCHOOL DISTRICT",
"GREEN ISLAND UNION FREE SCHOOL DISTRICT",
"GREENE CENTRAL SCHOOL DISTRICT",
"GREENWICH CENTRAL SCHOOL DISTRICT",
"GROTON CENTRAL SCHOOL DISTRICT",
"GUILDERLAND CENTRAL SCHOOL DISTRICT",
"HALF HOLLOW HILLS CENTRAL SCHOOL DISTRICT",
"HAMBURG CENTRAL SCHOOL DISTRICT",
"HAMMOND CENTRAL SCHOOL DISTRICT",
"HANCOCK CENTRAL SCHOOL DISTRICT",
"HANNIBAL CENTRAL SCHOOL DISTRICT",
"HARPURSVILLE CENTRAL SCHOOL DISTRICT",
"HARRISON CENTRAL SCHOOL DISTRICT",
"HARRISVILLE CENTRAL SCHOOL DISTRICT",
"HARTFORD CENTRAL SCHOOL DISTRICT",
"HAUPPAUGE UNION FREE SCHOOL DISTRICT",
"HENDRICK HUDSON CENTRAL SCHOOL DISTRICT",
"HERKIMER CENTRAL SCHOOL DISTRICT",
"HIGHLAND CENTRAL SCHOOL DISTRICT",
"HIGHLAND FALLS CENTRAL SCHOOL DISTRICT",
"HILTON CENTRAL SCHOOL DISTRICT",
"HINSDALE CENTRAL SCHOOL DISTRICT",
"HOLLAND CENTRAL SCHOOL DISTRICT",
"HOLLAND PATENT CENTRAL SCHOOL DISTRICT",
"HOLLEY CENTRAL SCHOOL DISTRICT",
"HOMER CENTRAL SCHOOL DISTRICT",
"HONEOYE CENTRAL SCHOOL DISTRICT",
"HONEOYE FALLS-LIMA CENTRAL SCHOOL DISTRICT",
"HOOSICK FALLS CENTRAL SCHOOL DISTRICT",
"HORNELL CITY SCHOOL DISTRICT",
"HUDSON CITY SCHOOL DISTRICT",
"HUDSON FALLS CENTRAL SCHOOL DISTRICT",
"HUNTINGTON UNION FREE SCHOOL DISTRICT",
"HYDE PARK CENTRAL SCHOOL DISTRICT",
"IROQUOIS CENTRAL SCHOOL DISTRICT",
"ITHACA CITY SCHOOL DISTRICT",
"JAMESTOWN CITY SCHOOL DISTRICT",
"JAMESVILLE-DEWITT CENTRAL SCHOOL DISTRICT",
"JOHNSBURG CENTRAL SCHOOL DISTRICT",
"JOHNSTOWN CITY SCHOOL DISTRICT",
"KEENE CENTRAL SCHOOL DISTRICT",
"KINDERHOOK CENTRAL SCHOOL DISTRICT",
"KINGS PARK CENTRAL SCHOOL DISTRICT",
"KINGSTON CITY SCHOOL DISTRICT",
"LACKAWANNA CITY SCHOOL DISTRICT",
"LAKE GEORGE CENTRAL SCHOOL DISTRICT",
"LAKE PLACID CENTRAL SCHOOL DISTRICT",
"LAKELAND CENTRAL SCHOOL DISTRICT",
"LANSING CENTRAL SCHOOL DISTRICT",
"LAURENS CENTRAL SCHOOL DISTRICT",
"LAWRENCE UNION FREE SCHOOL DISTRICT",
"LE ROY CENTRAL SCHOOL DISTRICT",
"LEVITTOWN UNION FREE SCHOOL DISTRICT",
"LIBERTY CENTRAL SCHOOL DISTRICT",
"LINDENHURST UNION FREE SCHOOL DISTRICT",
"LITTLE FALLS CITY SCHOOL DISTRICT",
"LIVINGSTON MANOR CENTRAL SCHOOL DISTRICT",
"LIVONIA CENTRAL SCHOOL DISTRICT",
"LOCKPORT CITY SCHOOL DISTRICT",
"LONG BEACH CITY SCHOOL DISTRICT",
"LONGWOOD CENTRAL SCHOOL DISTRICT",
"LYME CENTRAL SCHOOL DISTRICT",
"LYNBROOK UNION FREE SCHOOL DISTRICT",
"LYONS CENTRAL SCHOOL DISTRICT",
"MADISON CENTRAL SCHOOL DISTRICT",
"MADRID-WADDINGTON CENTRAL SCHOOL DISTRICT",
"MAHOPAC CENTRAL SCHOOL DISTRICT",
"MALONE CENTRAL SCHOOL DISTRICT",
"MALVERNE UNION FREE SCHOOL DISTRICT",
"MANHASSET UNION FREE SCHOOL DISTRICT",
"MARATHON CENTRAL SCHOOL DISTRICT",
"MARGARETVILLE CENTRAL SCHOOL DISTRICT",
"MARION CENTRAL SCHOOL DISTRICT",
"MASSENA CENTRAL SCHOOL DISTRICT",
"MATTITUCK-CUTCHOGUE UNION FREE SCHOOL DISTRICT",
"MAYFIELD CENTRAL SCHOOL DISTRICT",
"MEDINA CENTRAL SCHOOL DISTRICT",
"MEXICO CENTRAL SCHOOL DISTRICT",
"MIDDLE COUNTRY CENTRAL SCHOOL DISTRICT",
"MIDDLETOWN CITY SCHOOL DISTRICT",
"MILFORD CENTRAL SCHOOL DISTRICT",
"MILLBROOK CENTRAL SCHOOL DISTRICT",
"MINEOLA UNION FREE SCHOOL DISTRICT",
"MINERVA CENTRAL SCHOOL DISTRICT",
"MORIAH CENTRAL SCHOOL DISTRICT",
"MORRIS CENTRAL SCHOOL DISTRICT",
"MOUNT MARKHAM CENTRAL SCHOOL DISTRICT",
"MOUNT SINAI UNION FREE SCHOOL DISTRICT",
"NAPLES CENTRAL SCHOOL DISTRICT",
"NEW PALTZ CENTRAL SCHOOL DISTRICT",
"NEW YORK MILLS UNION FREE SCHOOL DISTRICT",
"NEWARK CENTRAL SCHOOL DISTRICT",
"NEWARK VALLEY CENTRAL SCHOOL DISTRICT",
"NEWBURGH CITY SCHOOL DISTRICT",
"NEWCOMB CENTRAL SCHOOL DISTRICT",
"NEWFANE CENTRAL SCHOOL DISTRICT",
"NEWFIELD CENTRAL SCHOOL DISTRICT",
"NORTH BABYLON UNION FREE SCHOOL DISTRICT",
"NORTH COLLINS CENTRAL SCHOOL DISTRICT",
"NORTH ROSE-WOLCOTT CENTRAL SCHOOL DISTRICT",
"NORTH SALEM CENTRAL SCHOOL DISTRICT",
"NORTH SHORE CENTRAL SCHOOL DISTRICT",
"NORTH SYRACUSE CENTRAL SCHOOL DISTRICT",
"NORTH TONAWANDA CITY SCHOOL DISTRICT",
"NORTH WARREN CENTRAL SCHOOL DISTRICT",
"NORTHEAST CENTRAL SCHOOL DISTRICT",
"NORTHEASTERN CLINTON CENTRAL SCHOOL DISTRICT",
"NORTHERN ADIRONDACK CENTRAL SCHOOL DISTRICT",
"NORTHVILLE CENTRAL SCHOOL DISTRICT",
"NORWICH CITY SCHOOL DISTRICT",
"NYACK UNION FREE SCHOOL DISTRICT",
"OGDENSBURG CITY SCHOOL DISTRICT",
"OLEAN CITY SCHOOL DISTRICT",
"ONEIDA CITY SCHOOL DISTRICT",
"ONEONTA CITY SCHOOL DISTRICT",
"ONONDAGA CENTRAL SCHOOL DISTRICT",
"OSSINING UNION FREE SCHOOL DISTRICT",
"OSWEGO CITY SCHOOL DISTRICT",
"OTEGO-UNADILLA CENTRAL SCHOOL DISTRICT",
"OWEGO-APALACHIN CENTRAL SCHOOL DISTRICT",
"PALMYRA-MACEDON CENTRAL SCHOOL DISTRICT",
"PANAMA CENTRAL SCHOOL DISTRICT",
"PATCHOGUE-MEDFORD UNION FREE SCHOOL DISTRICT",
"PAVILION CENTRAL SCHOOL DISTRICT",
"PAWLING CENTRAL SCHOOL DISTRICT",
"PEEKSKILL CITY SCHOOL DISTRICT",
"PENN YAN CENTRAL SCHOOL DISTRICT",
"PERRY CENTRAL SCHOOL DISTRICT",
"PERU CENTRAL SCHOOL DISTRICT",
"PINE BUSH CENTRAL SCHOOL DISTRICT",
"PLAINEDGE UNION FREE SCHOOL DISTRICT",
"PLAINVIEW-OLD BETHPAGE CENTRAL SCHOOL DISTRICT",
"PLEASANTVILLE UNION FREE SCHOOL DISTRICT",
"POLAND CENTRAL SCHOOL DISTRICT",
"PORT CHESTER-RYE UNION FREE SCHOOL DISTRICT",
"PORT JERVIS CITY SCHOOL DISTRICT",
"PORTVILLE CENTRAL SCHOOL DISTRICT",
"POTSDAM CENTRAL SCHOOL DISTRICT",
"RANDOLPH CENTRAL SCHOOL DISTRICT",
"RED CREEK CENTRAL SCHOOL DISTRICT",
"RED HOOK CENTRAL SCHOOL DISTRICT",
"REMSEN CENTRAL SCHOOL DISTRICT",
"RIVERHEAD CENTRAL SCHOOL DISTRICT",
"ROCHESTER CITY SCHOOL DISTRICT",
"ROCKVILLE CENTRE UNION FREE SCHOOL DISTRICT",
"ROME CITY SCHOOL DISTRICT",
"ROMULUS CENTRAL SCHOOL DISTRICT",
"ROSCOE CENTRAL SCHOOL DISTRICT",
"ROSLYN UNION FREE SCHOOL DISTRICT",
"ROTTERDAM-MOHONASEN CENTRAL SCHOOL DISTRICT",
"ROXBURY CENTRAL SCHOOL DISTRICT",
"RUSH-HENRIETTA CENTRAL SCHOOL DISTRICT",
"RYE CITY SCHOOL DISTRICT",
"RYE NECK UNION FREE SCHOOL DISTRICT",
"SACHEM CENTRAL SCHOOL DISTRICT",
"SALAMANCA CITY SCHOOL DISTRICT",
"SALEM CENTRAL SCHOOL DISTRICT",
"SALMON RIVER CENTRAL SCHOOL DISTRICT",
"SAYVILLE UNION FREE SCHOOL DISTRICT",
"SCARSDALE UNION FREE SCHOOL DISTRICT",
"SCHALMONT CENTRAL SCHOOL DISTRICT",
"SCHENEVUS CENTRAL SCHOOL DISTRICT",
"SCHODACK CENTRAL SCHOOL DISTRICT",
"SCHROON LAKE CENTRAL SCHOOL DISTRICT",
"SCHUYLERVILLE CENTRAL SCHOOL DISTRICT",
"SHENENDEHOWA CENTRAL SCHOOL DISTRICT",
"SHERMAN CENTRAL SCHOOL DISTRICT",
"SHERRILL CITY SCHOOL DISTRICT",
"SHOREHAM-WADING RIVER CENTRAL SCHOOL DISTRICT",
"SIDNEY CENTRAL SCHOOL DISTRICT",
"SILVER CREEK CENTRAL SCHOOL DISTRICT",
"SODUS CENTRAL SCHOOL DISTRICT",
"SOMERS CENTRAL SCHOOL DISTRICT",
"SOUTH COLONIE CENTRAL SCHOOL DISTRICT",
"SOUTH COUNTRY CENTRAL SCHOOL DISTRICT",
"SOUTH GLENS FALLS CENTRAL SCHOOL DISTRICT",
"SOUTH JEFFERSON CENTRAL SCHOOL DISTRICT",
"SOUTH KORTRIGHT CENTRAL SCHOOL DISTRICT",
"SOUTH SENECA CENTRAL SCHOOL DISTRICT",
"SOUTHAMPTON UNION FREE SCHOOL DISTRICT",
"SOUTHERN CAYUGA CENTRAL SCHOOL DISTRICT",
"SPACKENKILL UNION FREE SCHOOL DISTRICT",
"SPENCER-VAN ETTEN CENTRAL SCHOOL DISTRICT",
"STAMFORD CENTRAL SCHOOL DISTRICT",
"SWEET HOME CENTRAL SCHOOL DISTRICT",
"SYRACUSE CITY SCHOOL DISTRICT",
"TACONIC HILLS CENTRAL SCHOOL DISTRICT",
"THOUSAND ISLANDS CENTRAL SCHOOL DISTRICT",
"THREE VILLAGE CENTRAL SCHOOL DISTRICT",
"TICONDEROGA CENTRAL SCHOOL DISTRICT",
"TIOGA CENTRAL SCHOOL DISTRICT",
"TONAWANDA CITY SCHOOL DISTRICT",
"TROY CITY SCHOOL DISTRICT",
"TRUMANSBURG CENTRAL SCHOOL DISTRICT",
"TUCKAHOE UNION FREE SCHOOL DISTRICT",
"TULLY CENTRAL SCHOOL DISTRICT",
"TUPPER LAKE CENTRAL SCHOOL DISTRICT",
"UNADILLA VALLEY CENTRAL SCHOOL DISTRICT",
"UTICA CITY SCHOOL DISTRICT",
"VALLEY STREAM CENTRAL HIGH SCHOOL DISTRICT",
"VESTAL CENTRAL SCHOOL DISTRICT",
"VICTOR CENTRAL SCHOOL DISTRICT",
"WALLKILL CENTRAL SCHOOL DISTRICT",
"WALTON CENTRAL SCHOOL DISTRICT",
"WANTAGH UNION FREE SCHOOL DISTRICT",
"WARRENSBURG CENTRAL SCHOOL DISTRICT",
"WARSAW CENTRAL SCHOOL DISTRICT",
"WARWICK VALLEY CENTRAL SCHOOL DISTRICT",
"WASHINGTONVILLE CENTRAL SCHOOL DISTRICT",
"WATERFORD-HALFMOON UNION FREE SCHOOL DISTRICT",
"WATERLOO CENTRAL SCHOOL DISTRICT",
"WATERTOWN CITY SCHOOL DISTRICT",
"WATERVLIET CITY SCHOOL DISTRICT",
"WATKINS GLEN CENTRAL SCHOOL DISTRICT",
"WAYNE CENTRAL SCHOOL DISTRICT",
"WEBSTER CENTRAL SCHOOL DISTRICT",
"WELLS CENTRAL SCHOOL DISTRICT",
"WEST CANADA VALLEY CENTRAL SCHOOL DISTRICT",
"WEST HEMPSTEAD UNION FREE SCHOOL DISTRICT",
"WEST IRONDEQUOIT CENTRAL SCHOOL DISTRICT",
"WESTBURY UNION FREE SCHOOL DISTRICT",
"WHITEHALL CENTRAL SCHOOL DISTRICT",
"WHITESBORO CENTRAL SCHOOL DISTRICT",
"WHITESVILLE CENTRAL SCHOOL DISTRICT",
"WILLIAM FLOYD UNION FREE SCHOOL DISTRICT",
"WILLIAMSON CENTRAL SCHOOL DISTRICT",
"WILSON CENTRAL SCHOOL DISTRICT",
"WINDSOR CENTRAL SCHOOL DISTRICT",
"WORCESTER CENTRAL SCHOOL DISTRICT",
"YONKERS CITY SCHOOL DISTRICT",
"YORK CENTRAL SCHOOL DISTRICT",
"YORKTOWN CENTRAL SCHOOL DISTRICT"]


In [0]:
districtcolumntolist=census_cleaned2.select(census_cleaned2.School_district).rdd.flatMap(lambda x: x).collect()

In [0]:
a = sorted(districtcolumntolist, reverse=False)
a[0]

Out[14]: 'ADDISON CENTRAL SCHOOL DISTRICT'

In [0]:
b = sorted(list(schools_we_have))
b[0]

Out[15]: 'ADDISON CENTRAL SCHOOL DISTRICT'

In [0]:
main_list = list(set(a) - set(b))

In [0]:
sorted(main_list)

Out[17]: ['ALBION CENTRAL SCHOOL DISTRICT',
 'ALDEN CENTRAL SCHOOL DISTRICT',
 'ALEXANDER CENTRAL SCHOOL',
 'ALFRED-ALMOND CENTRAL SCHOOL DISTRICT 1',
 'ALLEGANY-LIMESTONE CSD',
 'ALTMAR PARISH WILLIAMSTWN CENTRAL SCHOOL DISTRICT',
 'AMAGANSETT UNION FREE SCHOOL DISTRICT',
 'AMITYVILLE UF SCHOOL DISTRICT',
 'AMSTERDAM SCHOOL DISTRICT',
 'ANGELICA-BELMONT CENTRAL SCHOOL DISTRICT',
 'ARDSLEY UF SCHOOL DISTRICT',
 'AVERILL PARK CSD',
 'BAINBRIDGE GUILFORD CTL SCHOOL DISTRICT',
 'BALDWINSVILLE CTL SCHOOL DISTRICT',
 'BALLSTON SPA CTL SCHOOL DISTRICT',
 'BAYPORT-BLUE POINT UF SCHOOL DISTRICT',
 'BEEKMANTOWN CENTRAL SCHOOL',
 'BELLEVILLE HENDERSON CENTRAL SCHOOL',
 'BELLMORE UNION FREE SCHOOL DISTRICT',
 'BELLMORE-MERRICK CTL HIGH SCHOOL DISTRICT',
 'BEMUS PT CENTRAL SCHOOL DISTRICT',
 'BERNE KNOX-WESTERLO CENTRAL SCHOOL DISTRICT',
 'BETHLEHEM CENTRAL SCHOOL',
 'BETHPAGE UF SCHOOL DISTRICT',
 'BLIND BROOK-RYE UF SCHOOL DISTRICT',
 'BRENTWOOD UF SCHOOL DISTRICT',
 'BRIARCLIFF MANOR U F SCHOOL DISTRICT',
 'BRIDGEHAMPTON U F SCHOOL DISTRICT',
 'BROADALBIN-PERTH CENTRAL SCHOOL',
 'BRUNSWICK CENTRAL SCHOOL DISTRICT',
 'BRUSHTON MOIRA CTL SCHOOL',
 'BRYON BERGEN CENTRAL SCHOOL',
 'BURNT HILLS BALLSTON LAKE CENTRAL SCHOOL DISTRICT',
 'BYRUM HILLS-C.S.D.',
 'CALEDONIA-MUMFORD CTL SCHOOL DISTRICT',
 'CAMPBELL-SAVONA SCHOOL DISTRICT',
 'CANAJOHARIE CENTRAL SCHOOL',
 'CANANDAIGUA CTY SCHOOL DISTRICT',
 'CANISTEO-GREENWOOD CENTRAL SCHOOL DISTRICT',
 'CARLE PLACE UF SCHOOL DISTRICT',
 'CASSADAGA VAL CTL SCHOOL DISTRICT',
 'CATSKILL CTL SCHOOL DISTRICT',
 'CATTARAUGUS-LITTLE VALLEY CONSOLIDATED SCHOOL DISTRICT',
 'CAZENOVIA CENTRAL SCHOOL',
 'CENTER MORICHES UF SCHOOL DISTRICT',
 'CENTRAL ISLIP UF SCHOOL DISTRICT',
 'CENTRAL VALLEY CSD AT ILION-MOHAWK',
 'CHARLOTTE VALLEY CTL SCHOOL',
 'CHATEAUGAY CENTRAL SCHOOL',
 'CHAZY UNION FREE SCHOOL DISTRICT',
 'CHERRY VALLEY-SPRINGFIELD CENTRL SCHL DISTRICT',
 'CHESTER UF SCHOOL DISTRICT',
 'CHITTENANGO CTL SCHOOL DISTRICT',
 'CHURCHVILLE CHILI CTL SCHOOL DISTRICT',
 'CINCINNATUS CENTRAL SCHOOL',
 'CLEVELAND HILLS U F SCHOOL DISTRICT',
 'COBLESKILL-RICHMONDVILLE CENTRAL SCHOOL',
 'COLD SPRING HARBOR CENTRAL SCHOOL DISTRICT 2',
 'COMMACK UF SCHOOL DISTRICT 10',
 'CONNETQUOT CTL SCHOOL DISTRICT',
 'COPIAGUE UF SCHOOL DISTRICT',
 'COXSACKIE ATHENS CTL SCHOOL DISTRICT',
 'CROTON HARMON U F SCHOOL DISTRICT',
 'CROWN POINT CENTRAL SCHOOL',
 'CUBA-RUSHFORD SCHOOL DISTRICT',
 'DEER PARK UF SCHOOL DISTRICT',
 'DELHI CENTRAL SCHOOL DISTRICT',
 'DOLGEVILLE CENTRAL SCHOOL',
 'DOVER U F SCHOOL DISTRICT',
 'EAST HAMPTON UF SCHOOL DISTRICT 1',
 'EAST ISLIP UFSD',
 'EAST MORICHES UNION FREE SCHOOL DISTRICT',
 'EAST QUOGUE UF SCHOOL DISTRICT',
 'EAST RAMAPO CENTRAL SCHOOL DISTRICT',
 'EAST ROCHESTER UF SCHOOL DISTRICT',
 'EAST SYRACUSE-MINOA CENTRAL SCHOOL DISTRICT',
 'EAST WILLISTON UF SCHOOL DISTRICT',
 'EASTPORT-SOUTH MANOR CENTRAL SCHOOL DISTRICT',
 'EDGEMONT UF SCHOOL DISTRICT',
 'EDINBURG COMMON SCHOOL DISTRICT',
 'EDMESTON CTL SCHOOL DISTRICT',
 'ELIZABETHTOWN LEWIS CENTRAL SCHOOL DISTRICT',
 'ELLICOTTVILLE CENT SCHOOL',
 'ELMIRA HGTS CENTRAL SCHOOL',
 'ELMONT UNION FREE SCHOOL DISTRICT',
 'ELMSFORD UF SCHOOL DISTRICT',
 'ELWOOD UF SCHOOL DISTRICT',
 'EVANS BRANT CENTRAL SCHOOL DISTRICT',
 'FIRE ISLAND UNION FREE SCHOOL DISTRICT',
 'FLORAL PARK BELLEROSE UNION FREE SCHOOL DISTRICT',
 'FONDA FULTONVILLE CTL SCHOOL DISTRICT',
 'FORESTVILLE CENTRAL SCHOOL',
 'FORT EDWARD U F SCHOOL DISTRICT',
 'FORT PLAIN CENT SCHOOL DISTRICT',
 'FRANKFORT SCHUYLER CENTRAL SCHOOL DISTRICT',
 'FRANKLIN SQ UF SCHOOL DISTRICT',
 'FRANKLINVILLE CENTRAL SCHOOL DISTRICT 1',
 'FREDONIA CTL SCHOOL DISTRICT',
 'GARDEN CITY U F SCHOOL DISTRICT',
 'GARRISON U F SCHOOL DISTRICT',
 'GATES CHILI CTL SCHOOL DISTRICT',
 'GEN BROWN CENTRAL SCHOOL',
 'GERMANTOWN CENTRAL SCHOOL',
 'GILBERTSVILLE-MT UPTON CENTRAL SCHOOL DISTRICT',
 'GILBOA-CONESVILLE CTL SCHOOL DISTRICT',
 'GLENS FALLS COMMON SCHOOL DISTRICT',
 'GLOVERSVILLE CITY SCHOOL',
 'GORHAM-MIDDLE

In [0]:
len(main_list)

Out[18]: 328

In [0]:
# census_cleaned2 = census_cleaned2.withColumn('School_district', regexp_replace('School_district', 'ALDEN CENTRAL SCHOOL DISTRICT ', 'ALDEN CENTRAL SCHOOL DISTRICT'))
# census_cleaned2 = census_cleaned2.withColumn('School_district', regexp_replace('School_district', 'ALEXANDER CENTRAL SCHOOL', 'ALEXANDER CENTRAL SCHOOL DISTRICT'))
# census_cleaned2 = census_cleaned2.withColumn('School_district', regexp_replace('School_district', 'ALDEN CENTRAL SCHOOL DISTRICT ', 'ALDEN CENTRAL SCHOOL DISTRICT'))

In [0]:
name = spark.read.format('csv').options(header='true', inferschema='true').load("/mnt/julianngroglio/school-finance/name-changes")
display(name)

Census,Graduation,_c2,_c3
ALDEN CENTRAL SCHOOL DISTRICT,ALDEN CENTRAL SCHOOL DISTRICT,null,null
ALEXANDER CENTRAL SCHOOL,ALEXANDER CENTRAL SCHOOL DISTRICT,null,null
ALFRED-ALMOND CENTRAL SCHOOL DISTRICT 1,ALFRED-ALMOND CENTRAL SCHOOL DISTRICT,null,null
ALLEGANY-LIMESTONE CSD,ALLEGANY-LIMESTONE CENTRAL SCHOOL DISTRICT,null,null
ALTMAR PARISH WILLIAMSTWN CENTRAL SCHOOL DISTRICT,ALTMAR-PARISH-WILLIAMSTOWN CENTRAL SCHOOL DISTRICT,null,null
AMITYVILLE UF SCHOOL DISTRICT,AMITYVILLE UNION FREE SCHOOL DISTRICT,null,null
AMSTERDAM SCHOOL DISTRICT,AMSTERDAM CITY SCHOOL DISTRICT,null,null
ARDSLEY UF SCHOOL DISTRICT,ARDSLEY UNION FREE SCHOOL DISTRICT,null,null
BAINBRIDGE GUILFORD CTL SCHOOL DISTRICT,BAINBRIDGE-GUILFORD CENTRAL SCHOOL DISTRICT,null,null
BALDWINSVILLE CTL SCHOOL DISTRICT,BALDWINSVILLE CENTRAL SCHOOL DISTRICT,null,null


In [0]:
official_name =name.select(name.Graduation).rdd.flatMap(lambda x: x).collect()
# print(official_name)
# display(name)
current_name =name.select(name['Census ']).rdd.flatMap(lambda x: x).collect()

In [0]:
censusPD = census_cleaned2.toPandas()
display(censusPD)

StateID,School_district,Total_revenue_state,Total_revenue_from_local_sources,SPED_rev_state,COE_pupil_support,Property_tax_revenue,Total_salaries_and_wages,Total_revenue,Total_federal_revenue,FEDREV_math_science_teach_quality,STATEREV_staff_improvement_programs,STATEREV_gifted_and_talented_programs,Total_expenditure,Total_current_spending_on_programs,Current_spending_on_instruction,Total_expenditure_support_services,Expenditure_instructional_staff_support,Expenditure_gen_administration,Expenditure_school_administration
33,BUFFALO CITY SCHOOL DISTRICT,830603,145801,108671,36598,0,361669,1063839,87435,0,0,0,1002588,894051,596399,255087,33541,4528,42617
33,ROCHESTER CITY SCHOOL DISTRICT,698695,136369,96350,40601,0,398096,908105,73041,0,0,0,1084531,849263,548635,273442,29439,5076,48430
33,NEW YORK CITY SCHOOL DISTRICT,11852988,19976042,1571872,233803,0,12535679,33788513,1959483,0,0,0,33799629,30024166,23100440,6346293,266295,87476,1213363
33,SYRACUSE CITY SCHOOL DISTRICT,384640,76934,52520,20380,0,239781,515531,53957,0,0,0,561015,463993,298635,146957,19320,3229,21553
33,YONKERS CITY SCHOOL DISTRICT,392963,234118,47788,25555,0,337273,670050,42969,0,0,0,673848,621402,440768,167238,8600,2717,40162
33,BERNE KNOX-WESTERLO CENTRAL SCHOOL DISTRICT,11602,10098,1756,697,9764,10605,22329,629,0,0,0,22825,19785,12277,7142,744,649,895
33,BETHLEHEM CENTRAL SCHOOL,34069,67455,5320,5234,60826,52610,103128,1604,0,0,0,97197,86587,53141,32075,3772,1667,4963
33,RAVENA COEYMANS SELKIRK CENTRAL SCHOOL DISTRICT,21653,26748,4793,1960,22973,24785,49793,1392,0,0,0,51022,43408,28292,14296,1698,920,1363
33,COHOES CITY SCHOOL DISTRICT,26322,15794,3866,2170,13614,22796,45204,3088,0,0,0,44261,42528,28383,12718,1719,869,2074
33,SOUTH COLONIE CENTRAL SCHOOL DISTRICT,32321,69185,4717,3507,64173,62033,104825,3319,0,0,0,103093,97743,69559,26213,3359,1565,3866


In [0]:
censusPD.replace(to_replace=current_name, value=official_name, inplace=True)


In [0]:
census_cleaned2 = spark.createDataFrame(censusPD)

In [0]:
census_cleaned2 = census_cleaned2.where(census_cleaned2.School_district != 'NEW YORK CITY SCHOOL DISTRICT #1-32')

In [0]:
census_cleaned2 = census_cleaned2.dropDuplicates(['School_district'])

In [0]:
census_cleaned2 = census_cleaned2.drop(census_cleaned2.StateID)

In [0]:
display(census_cleaned2)

School_district,Total_revenue_state,Total_revenue_from_local_sources,SPED_rev_state,COE_pupil_support,Property_tax_revenue,Total_salaries_and_wages,Total_revenue,Total_federal_revenue,FEDREV_math_science_teach_quality,STATEREV_staff_improvement_programs,STATEREV_gifted_and_talented_programs,Total_expenditure,Total_current_spending_on_programs,Current_spending_on_instruction,Total_expenditure_support_services,Expenditure_instructional_staff_support,Expenditure_gen_administration,Expenditure_school_administration
HORSEHEADS CENTRAL SCHOOL DISTRICT,39213,39706,4780,4124,32667,40603,81806,2887,0,0,0,93263,76962,49641,25571,3013,2351,3408
GOWANDA CENTRAL SCHOOL DISTRICT,24521,5430,2403,1635,3992,17476,33734,3783,0,0,0,31007,29696,18091,10822,1135,1263,1040
MENANDS UNION FREE SCHOOL DISTRICT,1549,6918,191,232,6482,3652,8733,266,0,0,0,12946,6570,4030,2399,251,444,142
SILVER CREEK CENTRAL SCHOOL DISTRICT,18889,5845,2509,929,5130,13900,27105,2371,0,0,0,26482,22650,15251,6844,1221,572,707
BELFAST CENTRAL SCHOOL DISTRICT,8253,1818,820,309,1438,4862,10860,789,0,0,0,10375,8673,5420,2853,278,513,181
VOORHEESVILLE CENTRAL SCHOOL DISTRICT,8151,17172,1249,1232,16251,13151,25814,491,0,0,0,23850,22793,15808,6521,567,673,701
CUBA-RUSHFORD CENTRAL SCHOOL DISTRICT,15125,6119,1925,699,5115,10481,22579,1335,0,0,0,20944,19119,10650,7947,1589,695,553
FILLMORE CENTRAL SCHOOL DISTRICT,13238,2705,1092,500,2063,7892,16949,1006,0,0,0,14896,13435,8360,4805,1080,496,232
WEEDSPORT CENTRAL SCHOOL DISTRICT,11780,9840,0,687,6721,10224,22382,762,0,0,0,24184,16489,10815,5424,712,700,582
HINSDALE CENTRAL SCHOOL DISTRICT,7600,2219,551,439,1722,5372,10474,655,0,0,0,10841,9316,5820,3308,478,397,193


# Grad rate cleaning

In [0]:
gradDF = spark.read.format('csv').options(header='true', inferschema='true').load('/mnt/julianngroglio/school-finance/grad-rate')
display(gradDF)

report_school_year,aggregation_index,aggregation_type,aggregation_code,aggregation_name,entity_inactive_date,lea_beds,lea_name,nrc_code,nrc_desc,county_code,county_name,nyc_ind,boces_code,boces_name,membership_code,membership_key,membership_desc,subgroup_code,subgroup_name,enroll_cnt,grad_cnt,grad_pct,local_cnt,local_pct,reg_cnt,reg_pct,reg_adv_cnt,reg_adv_pct,non_diploma_credential_cnt,non_diploma_credential_pct,still_enr_cnt,still_enr_pct,ged_cnt,ged_pct,dropout_cnt,dropout_pct
2019-20,3,District,1.010001E10,ALBANY CITY SCHOOL DISTRICT,null,null,null,3,Urban-Suburban High Needs,1,ALBANY,0,190,ALBANY-SCHENECTADY-SCHOHARIE(CAPITAL REGION),9,154,2016 Total Cohort - 4 Year Outcome,1,All Students,722,508,0.7,29,0.04,347,0.48,132,0.18,8,0.01,110,0.15,8,0.01,88,0.12
2019-20,3,District,1.020104E10,BERNE-KNOX-WESTERLO CENTRAL SCHOOL DISTRICT,null,null,null,5,Average Needs,1,ALBANY,0,190,ALBANY-SCHENECTADY-SCHOHARIE(CAPITAL REGION),9,154,2016 Total Cohort - 4 Year Outcome,1,All Students,71,65,0.92,7,0.1,35,0.49,23,0.32,1,0.01,4,0.06,0,0.0,1,0.01
2019-20,3,District,1.030606E10,BETHLEHEM CENTRAL SCHOOL DISTRICT,null,null,null,6,Low Needs,1,ALBANY,0,190,ALBANY-SCHENECTADY-SCHOHARIE(CAPITAL REGION),9,154,2016 Total Cohort - 4 Year Outcome,1,All Students,408,383,0.94,2,0.0,110,0.27,271,0.66,5,0.01,8,0.02,0,0.0,12,0.03
2019-20,3,District,1.040206E10,RAVENA-COEYMANS-SELKIRK CENTRAL SCHOOL DISTRICT,null,null,null,5,Average Needs,1,ALBANY,0,190,ALBANY-SCHENECTADY-SCHOHARIE(CAPITAL REGION),9,154,2016 Total Cohort - 4 Year Outcome,1,All Students,129,114,0.88,3,0.02,61,0.47,50,0.39,2,0.02,5,0.04,0,0.0,8,0.06
2019-20,3,District,1.050001E10,COHOES CITY SCHOOL DISTRICT,null,null,null,3,Urban-Suburban High Needs,1,ALBANY,0,190,ALBANY-SCHENECTADY-SCHOHARIE(CAPITAL REGION),9,154,2016 Total Cohort - 4 Year Outcome,1,All Students,152,127,0.84,7,0.05,68,0.45,52,0.34,0,0.0,8,0.05,0,0.0,17,0.11
2019-20,3,District,1.060106E10,SOUTH COLONIE CENTRAL SCHOOL DISTRICT,null,null,null,5,Average Needs,1,ALBANY,0,190,ALBANY-SCHENECTADY-SCHOHARIE(CAPITAL REGION),9,154,2016 Total Cohort - 4 Year Outcome,1,All Students,375,350,0.93,16,0.04,114,0.3,220,0.59,1,0.0,15,0.04,0,0.0,9,0.02
2019-20,3,District,1.062306E10,NORTH COLONIE CSD,null,null,null,6,Low Needs,1,ALBANY,0,190,ALBANY-SCHENECTADY-SCHOHARIE(CAPITAL REGION),9,154,2016 Total Cohort - 4 Year Outcome,1,All Students,512,478,0.93,4,0.01,150,0.29,324,0.63,6,0.01,13,0.03,0,0.0,15,0.03
2019-20,3,District,1.070103E10,GREEN ISLAND UNION FREE SCHOOL DISTRICT,null,null,null,3,Urban-Suburban High Needs,1,ALBANY,0,190,ALBANY-SCHENECTADY-SCHOHARIE(CAPITAL REGION),9,154,2016 Total Cohort - 4 Year Outcome,1,All Students,28,24,0.86,7,0.25,11,0.39,6,0.21,0,0.0,1,0.04,0,0.0,3,0.11
2019-20,3,District,1.080206E10,GUILDERLAND CENTRAL SCHOOL DISTRICT,null,null,null,6,Low Needs,1,ALBANY,0,190,ALBANY-SCHENECTADY-SCHOHARIE(CAPITAL REGION),9,154,2016 Total Cohort - 4 Year Outcome,1,All Students,374,351,0.94,5,0.01,72,0.19,274,0.73,0,0.0,15,0.04,0,0.0,8,0.02
2019-20,3,District,1.100306E10,VOORHEESVILLE CENTRAL SCHOOL DISTRICT,null,null,null,6,Low Needs,1,ALBANY,0,190,ALBANY-SCHENECTADY-SCHOHARIE(CAPITAL REGION),9,154,2016 Total Cohort - 4 Year Outcome,1,All Students,97,93,0.96,5,0.05,31,0.32,57,0.59,0,0.0,2,0.02,0,0.0,2,0.02


In [0]:
gradDF = gradDF.drop('report_school_year','aggregation_index','aggregation_type','aggregation_code','entity_inactive_date','lea_beds','lea_name','nrc_code','county_code','nyc_id','boces_code','boces_code','boces_name','membership_code','membership_key','membership_desc','subgroup_code','subgroup_name','grad_pct','local_pct','reg_pct','reg_adv_pct','non_diploma_credential_pct','still_enr_pct','ged_pct','dropout_pct')
display(gradDF)

aggregation_name,nrc_desc,county_name,nyc_ind,enroll_cnt,grad_cnt,local_cnt,reg_cnt,reg_adv_cnt,non_diploma_credential_cnt,still_enr_cnt,ged_cnt,dropout_cnt
ALBANY CITY SCHOOL DISTRICT,Urban-Suburban High Needs,ALBANY,0,722,508,29,347,132,8,110,8,88
BERNE-KNOX-WESTERLO CENTRAL SCHOOL DISTRICT,Average Needs,ALBANY,0,71,65,7,35,23,1,4,0,1
BETHLEHEM CENTRAL SCHOOL DISTRICT,Low Needs,ALBANY,0,408,383,2,110,271,5,8,0,12
RAVENA-COEYMANS-SELKIRK CENTRAL SCHOOL DISTRICT,Average Needs,ALBANY,0,129,114,3,61,50,2,5,0,8
COHOES CITY SCHOOL DISTRICT,Urban-Suburban High Needs,ALBANY,0,152,127,7,68,52,0,8,0,17
SOUTH COLONIE CENTRAL SCHOOL DISTRICT,Average Needs,ALBANY,0,375,350,16,114,220,1,15,0,9
NORTH COLONIE CSD,Low Needs,ALBANY,0,512,478,4,150,324,6,13,0,15
GREEN ISLAND UNION FREE SCHOOL DISTRICT,Urban-Suburban High Needs,ALBANY,0,28,24,7,11,6,0,1,0,3
GUILDERLAND CENTRAL SCHOOL DISTRICT,Low Needs,ALBANY,0,374,351,5,72,274,0,15,0,8
VOORHEESVILLE CENTRAL SCHOOL DISTRICT,Low Needs,ALBANY,0,97,93,5,31,57,0,2,0,2


In [0]:
gradDF = gradDF.withColumnRenamed("aggregation_name","School_district")
gradDF_clean = gradDF.withColumnRenamed("nrc_desc","Needs")
gradDF_clean = gradDF_clean.withColumnRenamed("enroll_cnt","Enrollment_count")
gradDF_clean = gradDF_clean.withColumnRenamed("grad_cnt","Graduation_count")
gradDF_clean = gradDF_clean.withColumnRenamed("reg_cnt","Regular_degree_count")
gradDF_clean = gradDF_clean.withColumnRenamed("reg_adv_cnt","Advanced_degree_count")
gradDF_clean = gradDF_clean.withColumnRenamed("non_diploma_credential_cnt","Non_diploma_count")
gradDF_clean = gradDF_clean.withColumnRenamed("county_name","County_name")
gradDF_clean = gradDF_clean.drop('nyc_ind')

In [0]:
gradDF_clean = gradDF_clean.dropna(how="any")
gradDF_clean = gradDF_clean.where(gradDF_clean.Graduation_count !='-')

In [0]:
#display(gradDF_clean)

In [0]:
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 1')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 2')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 3')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 4')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 5')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 6')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 7')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 8')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 9')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #10')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #11')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #12')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #13')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #14')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #15')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #16')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #17')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #18')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #19')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #20')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #21')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #22')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #23')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #24')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #25')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #26')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #27')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #28')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #29')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #30')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #31')
gradDF_clean = gradDF_clean.where(gradDF_clean.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #32')
display(gradDF_clean)


School_district,Needs,County_name,Enrollment_count,Graduation_count,local_cnt,Regular_degree_count,Advanced_degree_count,Non_diploma_count,still_enr_cnt,ged_cnt,dropout_cnt
ALBANY CITY SCHOOL DISTRICT,Urban-Suburban High Needs,ALBANY,722,508,29,347,132,8,110,8,88
BERNE-KNOX-WESTERLO CENTRAL SCHOOL DISTRICT,Average Needs,ALBANY,71,65,7,35,23,1,4,0,1
BETHLEHEM CENTRAL SCHOOL DISTRICT,Low Needs,ALBANY,408,383,2,110,271,5,8,0,12
RAVENA-COEYMANS-SELKIRK CENTRAL SCHOOL DISTRICT,Average Needs,ALBANY,129,114,3,61,50,2,5,0,8
COHOES CITY SCHOOL DISTRICT,Urban-Suburban High Needs,ALBANY,152,127,7,68,52,0,8,0,17
SOUTH COLONIE CENTRAL SCHOOL DISTRICT,Average Needs,ALBANY,375,350,16,114,220,1,15,0,9
NORTH COLONIE CSD,Low Needs,ALBANY,512,478,4,150,324,6,13,0,15
GREEN ISLAND UNION FREE SCHOOL DISTRICT,Urban-Suburban High Needs,ALBANY,28,24,7,11,6,0,1,0,3
GUILDERLAND CENTRAL SCHOOL DISTRICT,Low Needs,ALBANY,374,351,5,72,274,0,15,0,8
VOORHEESVILLE CENTRAL SCHOOL DISTRICT,Low Needs,ALBANY,97,93,5,31,57,0,2,0,2


In [0]:
demoDF = spark.read.format('csv').options(header='true', inferschema='true').load("/mnt/julianngroglio/school-finance/demographic-factors")
display(demoDF)

ENTITY_CD,ENTITY_NAME,YEAR,NUM_ELL,PER_ELL,NUM_AM_IND,PER_AM_IND,NUM_BLACK,PER_BLACK,NUM_HISP,PER_HISP,NUM_ASIAN,PER_ASIAN,NUM_WHITE,PER_WHITE,NUM_Multi,PER_Multi,NUM_SWD,PER_SWD,NUM_FEMALE,PER_FEMALE,NUM_MALE,PER_MALE,NUM_ECDIS,PER_ECDIS,NUM_MIGRANT,PER_MIGRANT,NUM_HOMELESS,PER_HOMELESS,NUM_FOSTER,PER_FOSTER,NUM_ARMED,PER_ARMED
4.605E11,FULTON CITY SCHOOL DISTRICT,2019,42,1,10,0,49,1,173,5,20,1,2985,90,80,2,605,18,1597,48,1720,52,2054,62,25,1,117,4,30,1,0,0
4.60701E11,HANNIBAL CENTRAL SCHOOL DISTRICT,2019,5,0,3,0,8,1,26,2,3,0,1219,96,14,1,198,16,612,48,661,52,951,75,0,0,21,2,2,0,0,0
4.60801E11,CENTRAL SQUARE CENTRAL SCHOOL DISTRICT,2019,0,0,16,0,30,1,72,2,18,0,3454,94,70,2,498,14,1781,49,1879,51,1710,47,0,0,20,1,4,0,4,0
4.60901E11,MEXICO CENTRAL SCHOOL DISTRICT,2019,4,0,5,0,11,1,43,2,4,0,1810,95,34,2,323,17,929,49,978,51,1066,56,0,0,12,1,9,0,8,0
4.613E11,OSWEGO CITY SCHOOL DISTRICT,2019,61,2,9,0,62,2,173,5,43,1,3339,90,85,2,646,17,1759,47,1952,53,2131,57,13,0,66,2,1,0,7,0
4.61801E11,PULASKI CENTRAL SCHOOL DISTRICT,2019,0,0,2,0,3,0,12,1,6,1,935,96,11,1,142,15,479,49,490,51,510,53,0,0,27,3,1,0,0,0
4.61901E11,SANDY CREEK CENTRAL SCHOOL DISTRICT,2019,1,0,5,1,5,1,15,2,5,1,735,94,13,2,101,13,365,47,413,53,455,58,1,0,7,1,1,0,9,1
4.62001E11,PHOENIX CENTRAL SCHOOL DISTRICT,2019,1,0,4,0,11,1,24,1,6,0,1618,94,52,3,193,11,840,49,875,51,947,55,0,0,35,2,2,0,6,0
4.70202E11,GILBERTSVILLE-MOUNT UPTON CENTRAL SCHOOL DISTRICT,2019,1,0,0,0,1,0,5,2,0,0,319,96,8,2,65,20,176,53,157,47,178,53,1,0,0,0,0,0,0,0
4.70501E11,EDMESTON CENTRAL SCHOOL DISTRICT,2019,0,0,0,0,2,1,5,1,6,2,352,95,5,1,42,11,184,50,186,50,163,44,1,0,1,0,0,0,0,0


In [0]:
demoDF = demoDF.drop('YEAR','NUM_ELL', 'PER_ELL', 'NUM_Multi', 'PER_Multi', 'NUM_MIGRANT', 'PER_MIGRANT', 'NUM_FOSTER', 'PER_FOSTER', 'NUM_ARMED', 'PER_ARMED', 'ENTITY_CD')

In [0]:
#display(demoDF)

In [0]:
demoDF = demoDF.withColumn('PER_HISP', col('PER_HISP') / 100);

In [0]:
demoDF = demoDF.withColumn('PER_HISP', col('PER_HISP') / 100);
demoDF = demoDF.withColumn('PER_BLACK', col('PER_BLACK') / 100);
demoDF = demoDF.withColumn('PER_AM_IND', col('PER_AM_IND') / 100);
demoDF = demoDF.withColumn('PER_ASIAN', col('PER_ASIAN') / 100);
demoDF = demoDF.withColumn('PER_WHITE', col('PER_WHITE') / 100);
demoDF = demoDF.withColumn('PER_SWD', col('PER_SWD') / 100);
demoDF = demoDF.withColumn('PER_FEMALE', col('PER_FEMALE') / 100);
demoDF = demoDF.withColumn('PER_MALE', col('PER_MALE') / 100);
demoDF = demoDF.withColumn('PER_ECDIS', col('PER_ECDIS') / 100);
demoDF = demoDF.withColumn('PER_HOMELESS', col('PER_HOMELESS') / 100);
display(demoDF)

ENTITY_NAME,NUM_AM_IND,PER_AM_IND,NUM_BLACK,PER_BLACK,NUM_HISP,PER_HISP,NUM_ASIAN,PER_ASIAN,NUM_WHITE,PER_WHITE,NUM_SWD,PER_SWD,NUM_FEMALE,PER_FEMALE,NUM_MALE,PER_MALE,NUM_ECDIS,PER_ECDIS,NUM_HOMELESS,PER_HOMELESS
FULTON CITY SCHOOL DISTRICT,10,0.0,49,0.01,173,5.0E-4,20,0.01,2985,0.9,605,0.18,1597,0.48,1720,0.52,2054,0.62,117,0.04
HANNIBAL CENTRAL SCHOOL DISTRICT,3,0.0,8,0.01,26,2.0E-4,3,0.0,1219,0.96,198,0.16,612,0.48,661,0.52,951,0.75,21,0.02
CENTRAL SQUARE CENTRAL SCHOOL DISTRICT,16,0.0,30,0.01,72,2.0E-4,18,0.0,3454,0.94,498,0.14,1781,0.49,1879,0.51,1710,0.47,20,0.01
MEXICO CENTRAL SCHOOL DISTRICT,5,0.0,11,0.01,43,2.0E-4,4,0.0,1810,0.95,323,0.17,929,0.49,978,0.51,1066,0.56,12,0.01
OSWEGO CITY SCHOOL DISTRICT,9,0.0,62,0.02,173,5.0E-4,43,0.01,3339,0.9,646,0.17,1759,0.47,1952,0.53,2131,0.57,66,0.02
PULASKI CENTRAL SCHOOL DISTRICT,2,0.0,3,0.0,12,1.0E-4,6,0.01,935,0.96,142,0.15,479,0.49,490,0.51,510,0.53,27,0.03
SANDY CREEK CENTRAL SCHOOL DISTRICT,5,0.01,5,0.01,15,2.0E-4,5,0.01,735,0.94,101,0.13,365,0.47,413,0.53,455,0.58,7,0.01
PHOENIX CENTRAL SCHOOL DISTRICT,4,0.0,11,0.01,24,1.0E-4,6,0.0,1618,0.94,193,0.11,840,0.49,875,0.51,947,0.55,35,0.02
GILBERTSVILLE-MOUNT UPTON CENTRAL SCHOOL DISTRICT,0,0.0,1,0.0,5,2.0E-4,0,0.0,319,0.96,65,0.2,176,0.53,157,0.47,178,0.53,0,0.0
EDMESTON CENTRAL SCHOOL DISTRICT,0,0.0,2,0.01,5,1.0E-4,6,0.02,352,0.95,42,0.11,184,0.5,186,0.5,163,0.44,1,0.0


In [0]:
demoDF = demoDF.withColumnRenamed("ENTITY_NAME","School_district")
demoDF = demoDF.withColumnRenamed("NUM_SWD","NUM_DISABILITY")
demoDF = demoDF.withColumnRenamed("PER_SWD","PER_DISABILITY")
demoDF = demoDF.withColumnRenamed("NUM_ECDIS","NUM_ECONOMIC_DISADV")
demoDF = demoDF.withColumnRenamed("PER_ECDIS","PER_ECONOMIC_DISADV")
demoDF = demoDF.withColumnRenamed("NUM_AM_IND","NUM_NATIVE")
demoDF = demoDF.withColumnRenamed("PER_AM_IND","PER_NATIVE")
display(demoDF)

School_district,NUM_NATIVE,PER_NATIVE,NUM_BLACK,PER_BLACK,NUM_HISP,PER_HISP,NUM_ASIAN,PER_ASIAN,NUM_WHITE,PER_WHITE,NUM_DISABILITY,PER_DISABILITY,NUM_FEMALE,PER_FEMALE,NUM_MALE,PER_MALE,NUM_ECONOMIC_DISADV,PER_ECONOMIC_DISADV,NUM_HOMELESS,PER_HOMELESS
FULTON CITY SCHOOL DISTRICT,10,0.0,49,0.01,173,5.0E-4,20,0.01,2985,0.9,605,0.18,1597,0.48,1720,0.52,2054,0.62,117,0.04
HANNIBAL CENTRAL SCHOOL DISTRICT,3,0.0,8,0.01,26,2.0E-4,3,0.0,1219,0.96,198,0.16,612,0.48,661,0.52,951,0.75,21,0.02
CENTRAL SQUARE CENTRAL SCHOOL DISTRICT,16,0.0,30,0.01,72,2.0E-4,18,0.0,3454,0.94,498,0.14,1781,0.49,1879,0.51,1710,0.47,20,0.01
MEXICO CENTRAL SCHOOL DISTRICT,5,0.0,11,0.01,43,2.0E-4,4,0.0,1810,0.95,323,0.17,929,0.49,978,0.51,1066,0.56,12,0.01
OSWEGO CITY SCHOOL DISTRICT,9,0.0,62,0.02,173,5.0E-4,43,0.01,3339,0.9,646,0.17,1759,0.47,1952,0.53,2131,0.57,66,0.02
PULASKI CENTRAL SCHOOL DISTRICT,2,0.0,3,0.0,12,1.0E-4,6,0.01,935,0.96,142,0.15,479,0.49,490,0.51,510,0.53,27,0.03
SANDY CREEK CENTRAL SCHOOL DISTRICT,5,0.01,5,0.01,15,2.0E-4,5,0.01,735,0.94,101,0.13,365,0.47,413,0.53,455,0.58,7,0.01
PHOENIX CENTRAL SCHOOL DISTRICT,4,0.0,11,0.01,24,1.0E-4,6,0.0,1618,0.94,193,0.11,840,0.49,875,0.51,947,0.55,35,0.02
GILBERTSVILLE-MOUNT UPTON CENTRAL SCHOOL DISTRICT,0,0.0,1,0.0,5,2.0E-4,0,0.0,319,0.96,65,0.2,176,0.53,157,0.47,178,0.53,0,0.0
EDMESTON CENTRAL SCHOOL DISTRICT,0,0.0,2,0.01,5,1.0E-4,6,0.02,352,0.95,42,0.11,184,0.5,186,0.5,163,0.44,1,0.0


In [0]:
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 1')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 2')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 3')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 4')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 5')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 6')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 7')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 8')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT # 9')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #10')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #12')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #13')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #14')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #15')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #16')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #17')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #18')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #19')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #20')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #21')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #22')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #23')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #24')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #25')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #26')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #27')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #28')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #29')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #30')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #31')
demoDF = demoDF.where(demoDF.School_district != 'NEW YORK CITY GEOGRAPHIC DISTRICT #32')
display(demoDF)

School_district,NUM_NATIVE,PER_NATIVE,NUM_BLACK,PER_BLACK,NUM_HISP,PER_HISP,NUM_ASIAN,PER_ASIAN,NUM_WHITE,PER_WHITE,NUM_DISABILITY,PER_DISABILITY,NUM_FEMALE,PER_FEMALE,NUM_MALE,PER_MALE,NUM_ECONOMIC_DISADV,PER_ECONOMIC_DISADV,NUM_HOMELESS,PER_HOMELESS
FULTON CITY SCHOOL DISTRICT,10,0.0,49,0.01,173,5.0E-4,20,0.01,2985,0.9,605,0.18,1597,0.48,1720,0.52,2054,0.62,117,0.04
HANNIBAL CENTRAL SCHOOL DISTRICT,3,0.0,8,0.01,26,2.0E-4,3,0.0,1219,0.96,198,0.16,612,0.48,661,0.52,951,0.75,21,0.02
CENTRAL SQUARE CENTRAL SCHOOL DISTRICT,16,0.0,30,0.01,72,2.0E-4,18,0.0,3454,0.94,498,0.14,1781,0.49,1879,0.51,1710,0.47,20,0.01
MEXICO CENTRAL SCHOOL DISTRICT,5,0.0,11,0.01,43,2.0E-4,4,0.0,1810,0.95,323,0.17,929,0.49,978,0.51,1066,0.56,12,0.01
OSWEGO CITY SCHOOL DISTRICT,9,0.0,62,0.02,173,5.0E-4,43,0.01,3339,0.9,646,0.17,1759,0.47,1952,0.53,2131,0.57,66,0.02
PULASKI CENTRAL SCHOOL DISTRICT,2,0.0,3,0.0,12,1.0E-4,6,0.01,935,0.96,142,0.15,479,0.49,490,0.51,510,0.53,27,0.03
SANDY CREEK CENTRAL SCHOOL DISTRICT,5,0.01,5,0.01,15,2.0E-4,5,0.01,735,0.94,101,0.13,365,0.47,413,0.53,455,0.58,7,0.01
PHOENIX CENTRAL SCHOOL DISTRICT,4,0.0,11,0.01,24,1.0E-4,6,0.0,1618,0.94,193,0.11,840,0.49,875,0.51,947,0.55,35,0.02
GILBERTSVILLE-MOUNT UPTON CENTRAL SCHOOL DISTRICT,0,0.0,1,0.0,5,2.0E-4,0,0.0,319,0.96,65,0.2,176,0.53,157,0.47,178,0.53,0,0.0
EDMESTON CENTRAL SCHOOL DISTRICT,0,0.0,2,0.01,5,1.0E-4,6,0.02,352,0.95,42,0.11,184,0.5,186,0.5,163,0.44,1,0.0


In [0]:
#demoDF.printSchema()

In [0]:
%fs

ls /mnt/julianngroglio/school-finance

path,name,size,modificationTime
dbfs:/mnt/julianngroglio/school-finance/census-clean/,census-clean/,0,1660059215000
dbfs:/mnt/julianngroglio/school-finance/census-data,census-data,7255007,1659469854000
dbfs:/mnt/julianngroglio/school-finance/demo-clean/,demo-clean/,0,1660059217000
dbfs:/mnt/julianngroglio/school-finance/demographic-factors,demographic-factors,93703,1660055843000
dbfs:/mnt/julianngroglio/school-finance/grad-clean/,grad-clean/,0,1660059216000
dbfs:/mnt/julianngroglio/school-finance/grad-rate,grad-rate,166631,1659993188000
dbfs:/mnt/julianngroglio/school-finance/name-changes,name-changes,20054,1660155004000


In [0]:
census_cleaned2.write.option("header", "true").mode('overwrite').csv("/mnt/julianngroglio/school-finance/census-clean")
gradDF_clean.write.option("header", "true").mode('overwrite').csv("/mnt/julianngroglio/school-finance/grad-clean")
demoDF.write.option("header", "true").mode('overwrite').csv("/mnt/julianngroglio/school-finance/demo-clean")

In [0]:
%fs

ls /mnt/julianngroglio/school-finance

path,name,size,modificationTime
dbfs:/mnt/julianngroglio/school-finance/census-clean/,census-clean/,0,1660059215000
dbfs:/mnt/julianngroglio/school-finance/census-data,census-data,7255007,1659469854000
dbfs:/mnt/julianngroglio/school-finance/demo-clean/,demo-clean/,0,1660059217000
dbfs:/mnt/julianngroglio/school-finance/demographic-factors,demographic-factors,93703,1660055843000
dbfs:/mnt/julianngroglio/school-finance/grad-clean/,grad-clean/,0,1660059216000
dbfs:/mnt/julianngroglio/school-finance/grad-rate,grad-rate,166631,1659993188000
dbfs:/mnt/julianngroglio/school-finance/name-changes,name-changes,20054,1660155004000
